In [107]:
import pandas as pd
import numpy as np

In [12]:
# get model csv file
model = pd.read_csv("AD.model.csv")

I don't think these modules are that helpful

In [4]:
model['module'].unique()

array([nan, 'analysis', 'batch', 'clinical', 'curatedData',
       'demographics', 'experimentalData', 'immunoAssays', 'neuro', 'ngs',
       'qc', 'sageCommunity', 'tool'], dtype=object)

In [9]:
model.groupby(['module', 'Parent']).size().reset_index()

,module,Parent,0
0,analysis,DataProperty,1
1,analysis,analysisType,48
2,batch,DataProperty,9
3,clinical,DataProperty,11
4,clinical,apoeGenotype,6
5,clinical,mannerDeath,5
6,curatedData,DataProperty,1
7,curatedData,referenceSet,7
8,demographics,DataProperty,8
9,demographics,ageDeathUnits,6


Module should be assigned at the DataProperty level, and attributes that are valid values should inherit Module from their Parent. Probably all the manifest templates should get a module like "Template" or something like that.

In [19]:
# write a csv to edit with just the Data Property rows and DataType rows
edit_modules = model.loc[(model["Parent"] == "DataProperty") | (model["Parent"] == "DataType")]

In [20]:
edit_modules.to_csv("edit_modules.csv", index = False)

Read in manually edited modules.

In [4]:
updated_modules = pd.read_csv("edit_modules.csv")

Join back to original data model

In [35]:
parent_modules = updated_modules[["Attribute", "UpdatedModule"]]

In [114]:
joined_df = pd.merge(model, parent_modules, how = "left", on = "Attribute")
joined_df.head()

,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,columnType,module,UpdatedModule
0,Component,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,individual_human_metadata_template,SysBio AD individual human metadata template s...,NaN,"Component, individualID, individualIdSource, s...",NaN,False,DataType,NaN,sysbio.metadataTemplates-ad.individual.human,NaN,NaN,NaN,template
2,file_annotation_template,AD file annotation template schema,NaN,"Component, Filename, individualID, specimenID,...",NaN,False,DataType,NaN,sysbio.metadataTemplates-ad.manifest,NaN,NaN,NaN,template
3,biospecimen_metadata_template,SysBio AD biospecimen metadata template schema,NaN,"Component, individualID, specimenID, specimenI...",NaN,False,DataType,NaN,sysbio.metadataTemplates-ad.biospecimen,NaN,NaN,NaN,template
4,assay_scrnaSeq_metadata_template,SysBio scrnaSeq metadata template schema,NaN,"Component, specimenID, libraryID, assay, platf...",NaN,False,DataType,NaN,sysbio.metadataTemplates-assay.scrnaSeq,NaN,NaN,NaN,template


Make all the valid values inherit module from parent

In [162]:
#joined_df["UpdatedModule"] = joined_df["UpdatedModule"].fillna("")
new_df = joined_df.where(pd.notnull(joined_df), None)
new_df

,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,columnType,module,UpdatedModule
0,Component,None,None,None,NaN,True,None,NaN,None,NaN,None,None,None
1,individual_human_metadata_template,SysBio AD individual human metadata template s...,None,"Component, individualID, individualIdSource, s...",NaN,False,DataType,NaN,sysbio.metadataTemplates-ad.individual.human,NaN,None,None,template
2,file_annotation_template,AD file annotation template schema,None,"Component, Filename, individualID, specimenID,...",NaN,False,DataType,NaN,sysbio.metadataTemplates-ad.manifest,NaN,None,None,template
3,biospecimen_metadata_template,SysBio AD biospecimen metadata template schema,None,"Component, individualID, specimenID, specimenI...",NaN,False,DataType,NaN,sysbio.metadataTemplates-ad.biospecimen,NaN,None,None,template
4,assay_scrnaSeq_metadata_template,SysBio scrnaSeq metadata template schema,None,"Component, specimenID, libraryID, assay, platf...",NaN,False,DataType,NaN,sysbio.metadataTemplates-assay.scrnaSeq,NaN,None,None,template
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402,report,a document assembled by an author for the purp...,None,None,NaN,None,resourceType,NaN,http://purl.obolibrary.org/obo/IAO_0000088,NaN,string,sageCommunity,None
1403,tool,"Any file or link that represents a tool, model...",None,None,NaN,None,resourceType,NaN,Sage Bionetworks,NaN,string,sageCommunity,None
1404,ROSMAP-IA,The ROSMAP iPSC-derived Astrocyte Study (ROSMA...,None,None,NaN,None,study,NaN,https://adknowledgeportal.synapse.org/Explore/...,NaN,string,neuro,None
1405,entorhinal cortex,Component of the temporal lobe on the mesial s...,None,None,NaN,None,tissue,NaN,http://purl.obolibrary.org/obo/UBERON_0002728,NaN,string,experimentalData,None


In [117]:
new_df.dtypes

Attribute               object
Description             object
Valid Values            object
DependsOn               object
Properties             float64
Required                object
Parent                  object
DependsOn Component    float64
Source                  object
Validation Rules       float64
columnType              object
module                  object
UpdatedModule           object
dtype: object

In [163]:
updated_module_list = []
for i, row in new_df.iterrows():
    if row["UpdatedModule"] is None:
        if row["Parent"] is not None:
            updated_module_list.append(joined_df.loc[joined_df["Attribute"] == joined_df.iloc[i]["Parent"]]["UpdatedModule"].values[0,])
        else: updated_module_list.append(None)
    else:
        updated_module_list.append(row["UpdatedModule"])

In [161]:
type(joined_df.loc[joined_df["Attribute"] == joined_df.iloc[400]["Parent"]]["UpdatedModule"].values[0,])

str

In [164]:
new_df["FinalModules"] = updated_module_list

In [166]:
new_modules_df = new_df.drop(columns = ['module', 'UpdatedModule'])

In [169]:
new_modules_df = new_modules_df.rename(columns={"FinalModules": "module"})

Write as new version of the ad data model. Will need to re-pull changes to get the new study that Amelia added

In [170]:
new_modules_df.to_csv("AD.model.csv", index = False)